In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics

from rt_analysis import rt_filter_map_plot
import build_speedmaps_index

from IPython.display import display, Markdown
import pandas as pd
import geopandas as gpd
from siuba import *

from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    PROJECT_CRS,
    SEGMENT_GCS,
)

import shared_utils

In [2]:
## parameters cell
itp_id = 300

In [3]:
analysis_date = build_speedmaps_index.ANALYSIS_DATE
import datetime as dt
# if itp_id == 293:
#     analysis_date = dt.date(2023, 9, 13)

In [12]:
analysis_date = shared_utils.rt_dates.DATES['aug2024']

In [13]:
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_itp_id == itp_id)

FileNotFoundError: [Errno 2] No such file or directory: '_rt_progress_2024-08-14.parquet'

In [5]:
speedmap_index

,organization_name,organization_itp_id,organization_source_record_id,caltrans_district,_is_current,analysis_date,schedule_gtfs_dataset_key,status
53,City of Santa Monica,300,recJHFDLpGRMIFgnL,07 - Los Angeles,True,2024-04-17,efbbd5293be71f7a5de0cf82b59febe1,map_confirmed


In [6]:
SEGMENT_GCS

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/'

In [7]:
# speedmap_segs = gpd.read_parquet(f'{SEGMENT_GCS}segment_options/speedmap_segments_{analysis_date}.parquet')` #  non-aggregated

In [14]:
speedmap_segs = gpd.read_parquet(f'{SEGMENT_GCS}rollup_singleday/speeds_shape_speedmap_segments_{analysis_date}.parquet') #  aggregated

In [15]:
speedmap_segs.columns

Index(['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id', 'stop_pair',
       'stop_pair_name', 'segment_id', 'time_period', 'p50_mph', 'n_trips',
       'p20_mph', 'p80_mph', 'name', 'caltrans_district',
       'organization_source_record_id', 'organization_name', 'base64_url',
       'geometry'],
      dtype='object')

In [16]:
# speedmap_segs = speedmap_segs >> filter(_.schedule_gtfs_dataset_key == speedmap_index.schedule_gtfs_dataset_key.iloc[0])

In [17]:
speedmap_segs = speedmap_segs >> filter(_.name.str.contains('Torrance'))

In [18]:
speedmap_segs >> head(3)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,stop_pair,stop_pair_name,segment_id,time_period,p50_mph,n_trips,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry
214101,7cb3592df114dab1288176f1410563a1,8d78b7a45c70a6166d8a28940465c713,shp-3-01,870__242,Torrance Bl at Broadway__TORRANCE BL + PACIFIC...,870-242-1,offpeak,3.69,10,1.87,5.91,Torrance Schedule,07 - Los Angeles,recvzE9NXgGMmqcTH,City of Torrance,aHR0cHM6Ly90cmFuc2l0LnRvcnJhbmNlY2EuZ292L2hvbW...,"LINESTRING (-118.38611 33.83816, -118.38611 33..."
214102,7cb3592df114dab1288176f1410563a1,8d78b7a45c70a6166d8a28940465c713,shp-3-01,870__242,Torrance Bl at Broadway__TORRANCE BL + PACIFIC...,870-242-1,peak,4.33,14,3.26,7.16,Torrance Schedule,07 - Los Angeles,recvzE9NXgGMmqcTH,City of Torrance,aHR0cHM6Ly90cmFuc2l0LnRvcnJhbmNlY2EuZ292L2hvbW...,"LINESTRING (-118.38611 33.83816, -118.38611 33..."
214103,7cb3592df114dab1288176f1410563a1,8d78b7a45c70a6166d8a28940465c713,shp-3-01,870__242,Torrance Bl at Broadway__TORRANCE BL + PACIFIC...,870-242-1,all_day,4.22,24,2.91,7.16,Torrance Schedule,07 - Los Angeles,recvzE9NXgGMmqcTH,City of Torrance,aHR0cHM6Ly90cmFuc2l0LnRvcnJhbmNlY2EuZ292L2hvbW...,"LINESTRING (-118.38611 33.83816, -118.38611 33..."


In [19]:
speedmap_segs.time_period.unique()

array(['offpeak', 'peak', 'all_day'], dtype=object)

In [20]:
peak_test = speedmap_segs >> filter(_.time_period == 'peak')

In [21]:
time_periods = [(x, shared_utils.rt_utils.categorize_time_of_day(x)) for x in range(1, 25)]

In [22]:
time_periods

[(1, 'Owl'),
 (2, 'Owl'),
 (3, 'Owl'),
 (4, 'Early AM'),
 (5, 'Early AM'),
 (6, 'Early AM'),
 (7, 'AM Peak'),
 (8, 'AM Peak'),
 (9, 'AM Peak'),
 (10, 'Midday'),
 (11, 'Midday'),
 (12, 'Midday'),
 (13, 'Midday'),
 (14, 'Midday'),
 (15, 'PM Peak'),
 (16, 'PM Peak'),
 (17, 'PM Peak'),
 (18, 'PM Peak'),
 (19, 'PM Peak'),
 (20, 'Evening'),
 (21, 'Evening'),
 (22, 'Evening'),
 (23, 'Evening'),
 (24, 'Evening')]

In [23]:
total_peak = len([x[1] for x in time_periods if 'Peak' in x[1]])

In [24]:
total_peak

8

In [25]:
def prep_gdf(gdf):
    
    gdf = gdf.to_crs(calitp_data_analysis.geography_utils.CA_NAD83Albers)
    gdf['trips_per_hour'] = gdf.n_trips / total_peak

    ## shift to right side of road to display direction
    gdf.geometry = gdf.geometry.apply(shared_utils.rt_utils.try_parallel)
    gdf = gdf.apply(shared_utils.rt_utils.arrowize_by_frequency, axis=1)

    gdf = gdf >> arrange(_.trips_per_hour)

    return gdf

In [26]:
peak_test = prep_gdf(peak_test)

In [27]:
cmap = shared_utils.rt_utils.ZERO_THIRTY_COLORSCALE
url = shared_utils.rt_utils.SPEEDMAP_LEGEND_URL

In [28]:
speedmap_state = shared_utils.rt_utils.set_state_export(peak_test, filename='testnew5', map_type='speedmap',
                                      color_col='p20_mph', cmap=cmap, legend_url=url,
                                              cache_seconds=0, map_title=f'Speedmap Segs {analysis_date}')

writing to calitp-map-tiles/testing/testnew5.geojson.gz


In [29]:
speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs 2024-08-14',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/testnew5.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (33.85596962015502, -118.31386191576537),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIDIwMjQtMDgtMTQiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvdGVzdG5ldzUuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41LCAidG9vbHRpcF9zcGVlZF9rZXkiOiAicDIwX21waCJ9LCAidHlwZSI6ICJzcGVlZG1hcCJ9XSwgImxhdF9sb24iOiBbMzMuODU1OTY5NjIwMTU1MDIsIC0xMTguMzEzODYxOTE1NzY1MzddLCAiem9vbSI6IDEzLCAibGVnZW5kX3V

## TODO replace status with a scan of speedmap segs column?